![Header Image](../assets/header_image.png "Header Image")

Bienvenue dans l'exercice **Cartographie Sémantique par Grille basée sur Caméra**.

Dans cet exercice, nous allons utiliser des images prises par plusieurs caméras montées sur le véhicule pour obtenir une **vue aérienne (BEV) à 360°** de la route. Notre approche utilise la **transformation de perspective inverse (IPM)**.

Ensuite, nous appliquerons l'IPM à des images segmentées sémantiquement pour obtenir une **carte de grille sémantique** de l'environnement du véhicule, comme illustré dans l'image ci-dessous.

Les cartes de grille sémantiques sont similaires aux cartes de grille d'occupation et fournissent des informations supplémentaires sur le type d'objets présents dans l'environnement d'un véhicule.

![Example](ipm_assets/images/demo_carla.png "Example")

La carte de grille sémantique que nous allons obtenir peut faciliter les tâches de planification et de prédiction.

Dans cet exercice, nous allons parcourir les étapes suivantes :

- Charger les images et les paramètres de la caméra
- Utiliser OpenCV pour appliquer la transformation de perspective inverse
- Utiliser le modèle de caméra sténopé
- Appliquer les transformations de système de coordonnées
- Effectuer l'IPM
- Assembler plusieurs images en vue aérienne BEV

Commençons par importer tous les packages nécessaires pour cet exercice :

In [ ]:
import cv2
import numpy as np
import json
import matplotlib.pyplot as plt
import os

In [ ]:
def show_image(img, title=None):
    """this function shows an opencv image in this notebook"""
    rgb_img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    plt.imshow(rgb_img)
    if title is not None:
        plt.title(title)
    plt.show()

## Exemple des Entrées

Nous allons commencer cet exercice en appliquant **l'IPM à des images RGB** (non segmentées) capturées par des caméras montées sur le véhicule dans un simulateur (VTD). L'image chargée par le segment de code suivant a été capturée par la caméra avant droite du véhicule.

In [ ]:
# read image
img = cv2.imread("ipm_assets/images/vr_1.png")
# show image 
show_image(img, "Original Image")




## Tâche : Utiliser OpenCV pour effectuer l'IPM
Dans cette tâche, vous allez apprendre à utiliser OpenCV pour transformer une image, prise par une caméra, en une image de vue aérienne, comme illustré dans l'image ci-dessous.

![Expected Output](ipm_assets/images/expected_output_warp_persp.png "Expected Output")

Cette transformation peut être réalisée en utilisant une transformation homogène **P** (que nous appellerons `P_cam_to_bev` dans notre code).
Cette transformation peut mapper un point (en coordonnées d'image de caméra) vers des coordonnées routières (coordonnées BEV).

En utilisant les coordonnées **homogènes** **x<sub>c<sub>i</sub></sub>** pour l'image de caméra et **x<sub>r<sub>i</sub></sub>** pour la route, nous pouvons mapper les points **x<sub>c<sub>i</sub></sub>** vers les points **x<sub>r<sub>i</sub></sub>** en utilisant une multiplication matricielle.

La transformation homogène **P** est une **transformation projective** et peut être écrite comme une **matrice 3x3** :


$$ \large P =  \begin{bmatrix} p_{11} & p_{12} & p_{13} \\ p_{21} & p_{22} & p_{23}\\p_{31} & p_{32} & 1 \end{bmatrix}$$

Les points en coordonnées d'image sont mappés vers les coordonnées routières comme suit :

$$ \large \begin{bmatrix} w_i x_{r_i} \\ w_i y_{r_i} \\ w_i \end{bmatrix} = P \begin{bmatrix} x_{c_i} \\ y_{c_i} \\ 1 \end{bmatrix}$$


En utilisant l'équation ci-dessus et en connaissant **quatre points sources dans les coordonnées d'image** et leurs **quatre points correspondants dans les coordonnées routières**, nous pouvons résoudre un système d'équations pour les 8 paramètres inconnus de **P**.

Heureusement, OpenCV fournit une méthode pour faire cela !

En utilisant `cv2.getPerspectiveTransform()`, qui prend `source_coordinates` (**x<sub>c<sub>1</sub></sub>, x<sub>c<sub>2</sub></sub>, x<sub>c<sub>3</sub></sub>, x<sub>c<sub>4</sub></sub>**) et `destination_coordinates` (**x<sub>r<sub>1</sub></sub>, x<sub>r<sub>2</sub></sub>, x<sub>r<sub>3</sub></sub>, x<sub>r<sub>4</sub></sub>**) comme paramètres, nous pouvons calculer la matrice d'une transformation à partir des points sources fournis vers les points de destination. En interne, cette fonction applique l'élimination gaussienne (par défaut, d'autres méthodes se trouvent [ici](https://docs.opencv.org/master/d2/de8/group__core__array.html#gaaf9ea5dcc392d5ae04eacb9920b9674c)) pour calculer les éléments de la matrice de transformation de perspective.

Votre tâche est de fournir 4 points sources et 4 points de destination pour `getPerspectiveTransform()`.
Les points sources doivent être fournis en coordonnées d'image (x<sub>c<sub>i</sub></sub>,y<sub>c<sub>i</sub></sub>).

Une façon pratique d'y parvenir est de choisir les points de manière à ce qu'ils forment un rectangle en vue de dessus.

Ensuite, vous appliquerez la matrice de transformation résultante pour produire une image BEV en utilisant `cv2.warpPerpective()`.

Remplacez les espaces réservés `None` par votre code.

#### __Conseils__ :
- Les points sources et de destination doivent être fournis sous forme de listes Python
- (Uniquement pour **l'utilisation locale**, ne fonctionne pas dans JupyterHub) `cv2.show()` peut être utilisé pour afficher l'image d'entrée. Survolez l'image. La position actuelle de la souris en coordonnées d'image est affichée dans le coin inférieur gauche. Utilisez ceci pour obtenir les coordonnées des points sources. Vous pouvez augmenter le temps dans `cv2.waitKey()` (spécifié en millisecondes), qui définit la durée d'affichage du graphique. Utilisez `cv2.getPerspectiveTransform()` pour calculer la matrice de transformation de perspective inverse. Consultez la [documentation](https://docs.opencv.org/4.x/da/d54/group__imgproc__transform.html#ga20f62aa3235d869c9956436c870893ae) pour plus de détails.
- Utilisez `cv2.warpPerpective()` pour appliquer la matrice de transformation de perspective calculée à l'image d'entrée. Consultez la [documentation](https://docs.opencv.org/4.x/da/d54/group__imgproc__transform.html#gaf73673a7e8e18ec6963e3774e6a94b87) pour plus de détails.


In [ ]:
# size of output image
height, width = 600, 400

### START CODE HERE ###

# plot the input image
#cv2.imshow("Input image", img)
#cv2.waitKey(60)
#cv2.destroyAllWindows()

# define the source points
x_c_1, x_c_2, x_c_3, x_c_4 = None, None, None, None

src_pts = np.float32([x_c_1, x_c_2, x_c_3, x_c_4])
# define the destination points 
x_r_1, x_r_2, x_r_3, x_r_4 = None, None, None, None

dst_pts =np.float32([x_r_1, x_r_2, x_r_3, x_r_4])

# calculate the perspective transform matrix
P_cam_to_bev = None
#print (P_cam_to_bev)

# caculate the output image
output = None

### END CODE HERE ###
show_image(output, "Output Image")


In the following, we will draw a trapezoid connecting the source points in the image and apply the calculated transformation to the image again to see how the trapezoid is mapped. 

In [ ]:
# function to draw a line
def draw_line(image, pt1, pt2, color=(0,0,255), thickness=5):
    print(pt1, pt2)
    cv2.line(image, pt1.astype(int), pt2.astype(int), color, thickness)
    
# function to draw rectanges:
def draw_rectange(image, pts, color=(0,0,255), thickness=5):
    draw_line(image, pts[0], pts[1], color, thickness)
    draw_line(image, pts[1], pts[2], color, thickness)
    draw_line(image, pts[2], pts[3], color, thickness)
    draw_line(image, pts[3], pts[0], color, thickness)

In [ ]:
# draw a rectange that shows the source points
draw_rectange(img, src_pts)
# draw a rectangle that shows the destination points
#draw_rectange(output, dst_pts)
# caculate the output image
output =cv2.warpPerspective(img ,P_cam_to_bev, (width,height))


show_image(img, "Input Image")
show_image(output, "Output Image")

Cette méthode ne fonctionne pas uniquement pour la transformation d'images en vue aérienne. Elle peut transformer n'importe quelle vue en une autre, tant que les points sources et de destination sont coplanaires.

## IPM utilisant la Géométrie Projective
Comme la méthode utilisant OpenCV n'est pas exacte (car les points sources et de destination doivent être définis manuellement), une méthode basée sur la géométrie pour calculer l'image de vue aérienne en utilisant des transformations de système de coordonnées sera discutée.

La méthode basée sur la géométrie sera fondée sur un modèle de caméra simple et utilisera les matrices intrinsèques et extrinsèques pour calculer la matrice de transformation de perspective.

### Modèle de Caméra
Nous allons commencer par introduire le modèle de caméra simple que nous utiliserons dans cet exercice.
Ce modèle définit comment un point $\\textbf{X}=(X,Y,Z)$ dans le repère monde est transformé en un point $\\textbf{x}_{cam}=(x_c,y_c)$ dans l'image de la caméra.

Ce modèle utilise les paramètres intrinsèques et extrinsèques de la caméra.

#### Transformation du repère monde vers le repère caméra
Ici, nous **supposons** que le **repère monde** est le **repère principal du véhicule** puisque notre objectif est de produire une image de vue aérienne ou une carte de grille sémantique **centrée sur le véhicule**. Le repère de coordonnées principal du véhicule est souvent appelé le __base link__. Il peut être utilisé pour relier les coordonnées du véhicule aux coordonnées de la carte.

Nous définissons une transformation homogène **E** qui transforme les points du repère monde vers le repère caméra (toujours en trois dimensions) comme montré dans l'équation suivante :

$$ \large X_{cam} = E X $$


où **X** représente la coordonnée homogène du point (en trois dimensions) dans le repère monde (ici aussi le repère principal du véhicule) et **X<sub>cam</sub>** représente la coordonnée homogène du point (en trois dimensions) dans le repère caméra.

Cette matrice de transformation homogène incorpore les **paramètres extrinsèques** de la caméra.
Nous appellerons cette matrice la **matrice extrinsèque**.
Elle dépend de la pose de la caméra par rapport à un repère de référence. Dans notre cas, la matrice extrinsèque transforme le repère de coordonnées principal du véhicule vers le repère de coordonnées de la caméra.

**E** peut être composée d'une matrice de rotation 3x3 **R** et d'un vecteur de translation 3x1 **t**.
     

$$ \large E = \begin{bmatrix} R & t \\ 0^T & 1\end{bmatrix} = \begin{bmatrix} R & -R \tilde{C} \\ 0^T & 1\end{bmatrix}$$

Où $\tilde{C}$ est la coordonnée du repère dans le repère principal du véhicule.

Maintenant que nous pouvons mapper les points vers le repère caméra, nous allons les projeter dans le plan image.


#### Transformation du repère caméra vers le plan image
Nous définissons une transformation projective **K** qui transforme les points tridimensionnels dans les coordonnées du repère caméra en points bidimensionnels dans les coordonnées de l'image. Cette transformation peut également être définie comme une matrice, qui dépend des paramètres de la caméra, comme la distance focale $f$ et le point principal $p$. (Ici, nous ignorons la distorsion de l'objectif).
Nous appellerons cette matrice la **matrice intrinsèque**.
Les coordonnées homogènes d'un point tridimensionnel dans le repère caméra sont mappées vers les coordonnées homogènes d'un point bidimensionnel dans le plan image selon l'équation suivante :

$$ \large x_{cam} = K [I|0] X_{cam} = K \begin{bmatrix} 1 & 0 & 0 & 0\\ 0 & 1 & 0 & 0 \\ 0 & 0 & 1 & 0\end{bmatrix} X_{cam} $$

où **X<sub>cam</sub>** sont les coordonnées homogènes du point (tridimensionnel) dans le repère caméra et **x<sub>cam</sub>** est la coordonnée homogène du point (bidimensionnel) dans le plan caméra.


**K** peut être écrite comme suit :

$$ \large K = \begin{bmatrix} f_x & 0 & p_x \\ 0 & f_y & p_y \\ 0 & 0 & 1\end{bmatrix}$$


### Matrice de projection de la caméra
En utilisant les matrices de transformation intrinsèques et extrinsèques, nous pouvons définir la transformation qui mappe les points du système de coordonnées du véhicule vers le système de coordonnées du plan image, selon l'équation suivante :


$$ \large x_{cam} = P X$$

La matrice de projection **P** peut être calculée en multipliant **K** et **E** :

$$ \large P = K \begin{bmatrix} 1 & 0 & 0 & 0 \\ 0 & 1 & 0 & 0 \\ 0 & 0 & 1 & 0\end{bmatrix} E $$

La matrice $\large [I | 0]$ est nécessaire car nous n'avons besoin que des coordonnées __inhomogènes__ du point __X__ dans le repère monde.

## Tâche : Obtenir la matrice intrinsèque de la caméra
Les paramètres de la caméra peuvent être estimés en effectuant un étalonnage de la caméra.
Ici, l'étalonnage de la caméra a déjà été effectué et les paramètres sont fournis dans un fichier externe.
L'objectif de cette tâche est d'extraire ces paramètres du fichier externe.

Votre tâche ici est de lire la matrice intrinsèque à partir d'un fichier json.
Pour cela, vous allez implémenter une fonction qui obtient un chemin vers un fichier JSON et retourne un dictionnaire contenant les paramètres intrinsèques de toutes les caméras dans le fichier sous forme de **tableaux NumPy**.
Complétez la fonction `get_intrinsics()` en remplaçant les espaces réservés `None`.
#### __Conseils :__
- Inspectez le fichier `intrinsics.json` fourni sous `ipm_assets/cameras/intrinsics.json`.
- Utilisez `open()` de Python pour lire le fichier json. Consultez la [documentation](https://docs.python.org/3/library/functions.html#open).
- Utilisez la fonction `json.load()`. Consultez la [documentation](https://docs.python.org/3/library/json.html).
- après la lecture du fichier, vous devriez obtenir un dictionnaire qui contient la matrice intrinsèque de **8 caméras** montées sur un véhicule. Dans les fichiers json, les caméras sont nommées : **('vr_1', 'vl_1', 'hr_1', 'hl_1', 'vr_2', 'vl_2', 'hr_2', 'hl_2')**
- Notez que les matrices chargées sont sauvegardées sous forme de listes Python et doivent être converties en tableaux NumPy

In [ ]:
def get_intrinsics(f_path="ipm_assets/cameras/intrinsics.json"):
    ### START CODE HERE ###
    file = None
    intrinsics_dict = None
    for camera_n, intrinsic_matrix in intrinsics_dict.items():
        intrinsics_dict[None] = np.array(None)
    ### END CODE HERE ###
    return intrinsics_dict

intrinsics_dict = get_intrinsics()

The dictionary we loaded contains the intrinsic matrices of 8 different cameras. Let's print the intrinsic matrix of the **front right camera**.

In [ ]:
# Get the intrinsics of the first front facing camera (vr_1)
K = np.array(intrinsics_dict['vr_1'])
K

## Task: Calculate the camera extrinsic matrix
The goal of this task is to compute the extrinsic matrix of the camera.

Your task here is to read the extrinsic camera parameters from a JSON file.
The provided JSON files contain extrinsic information about multiple camera-mounted vehicles. 
This information contains the translation of each camera frame w.r.t the vehicle frame.
It also contains the orientation of each camera frame w.r.t the vehicle frame expressed in roll, pitch yaw angles.

The **position** and **orientation** of the camera need to be converted into a homogeneous transformation. 


The goal here is to implement a function that gets a path to a JSON file and returns a dictionary containing the homogeneous transformations describing the extrinsic parameters of all cameras in the file as NumPy arrays.

Replace the `None` placeholders with your code.
#### __Hints:__
- Read the translation vector (saved in the dictionary as 'translation') and convert it to a NumPy array.
- Read (roll, pitch, yaw) (saved in the dictionary as 'rotation_rpy').
- The representation (`roll`, `pitch`, `yaw`) is equivalent to (3x3)-rotation matrices performing the following operations in order:
    - a rotation around the x-axis with the roll angle
    - a rotation around the y-axis with the pitch angle
    - a rotation around the z-axis with the yaw angle
- Combine into a homogeneous transform (4x4)
    - use [`numpy.column_stack()`](https://numpy.org/doc/stable/reference/generated/numpy.column_stack.html) and [`numpy.row_stack()`](https://numpy.org/devdocs/reference/generated/numpy.row_stack.html)
    - Start by performing a column-wise stacking of the rotation matrix and the translation vector. The result should be a 3x4 matrix,
    - then perform a row-wise stacking of the resulting 3x4 matrix and the array `np.array([0., 0., 0., 1.])`. The result should be a 4x4 matrix. 
    - The result should be a dictionary that contains camera names as keys and their corresponding extrinsic matrices as values. 

In [ ]:
import json 

def get_extrinsics(f_path="ipm_assets/cameras/extrinsics_rpy.json"):
    ### START CODE HERE ###
    
    # read json file
    file = None
    extrinsics_rpy_dict = None

    # extrinsic dict (here we will save the extrinsic matrix for each camera)
    extrinsics_dict = dict()
    
    for camera_n, extrinsic_params in extrinsics_rpy_dict.items():
        # extract translation parameter
        t = None
        
        # get roll, pitch and yaw for the camera 'cmera_n'
        roll, pitch, yaw = None
        # compute rotation matrix
        Rz = None
        Ry = None
        Rx = None

        R = Rz.dot(Ry.dot(Rx))

        
        # combine translation (3x1) and roatation matrix (3x3) into a 4x4 homogeneous transform
        transform = None
        # add 1 row ([0., 0., 0., 1.]) to complete the transform 
        transform = None
        extrinsics_dict[camera_n] = None
        
        ### END CODE HERE ###
    return extrinsics_dict

extrinsics_dict = get_extrinsics()

Let's print the extrinsic matrix of the **front right camera**.

In [ ]:
# Get the extrinsics of the first front facing camera (vr_1)
E = extrinsics_dict['vr_1']
E


Great! We can now build the complete camera projection model.

## Task: Calculate the projection matrix

Calculate the camera projection matrix of the front right camera by replacing the `None` placeholder with your code.
#### __Hints__:

- The output should be a 3x4 matrix
- The extrinsic matrix is a 4x4 matrix where the intrinsic matrix is a 3x3 matrix, so don't forget the $[I|0]$ matrix
- Remember that we already saved the intrinsic and extrinsic matrices of the front right camera (in `K` and `E`) 



In [ ]:
### START CODE HERE ###
P = None
### END CODE HERE ###
P

## Task: Mapping from  the road (BEV image) coordinates to the vehicle's coordinates

Now we will define the mapping **M** from (two-dimensional) BEV image plane points (in homogeneous road coordinates) to (three-dimensional) world points (in homogeneous vehicle frame coordinates). 
In its simplest form, the mapping would have the following equation:

$$\large  M_{\text{2Dto3D}} = \begin{bmatrix} 1 & 0 & 0 \\ 0 & 1 & 0 \\ 0 & 0 & 0 \\ 0 & 0 & 1\end{bmatrix} $$


The equaion for $M_{\text{2Dto3D}}$ is based on the assumption that *z<sub>r</sub> = 0* (all points on the road plane have the height zero).

In addition to *z<sub>r</sub> = 0* , this equation also requires that the coordinate system of the road and the vehicle are colocated and have the same orientation. 
Since not all of these conditions are satisfied, we have to adjust M.
The figure below shows that the two frames are not colocated and don't have the same orientation.

![Expected Output](ipm_assets/images/road_to_vehicle.png "OpenCV cooridnates")


#### Paramètres

Définissons d'abord quelques paramètres pour le mapping **M**.
- `px_per_m` est la résolution et définit le nombre de pixels utilisés pour dessiner 1m. Cette résolution est utilisée verticalement et horizontalement. Dans notre cas, 1px correspond à 0.1m².
- `output_width` et `output_height` sont les dimensions de l'image BEV.
- `shift_x` et `shift_y` spécifient de combien l'origine du repère véhicule est décalée par rapport au repère route

In [ ]:
# parameters for ipm
# output resolution
px_per_m = 10 # number of pixels per meter
# output size
output_width = 798
output_height = 400
# shift to center of output image
shift_x = output_width/ 2.0 
shift_y = output_height / 2.0

### Ajouter l'opération de décalage :

Le premier ajustement que vous devrez apporter au mapping **M**, du repère de coordonnées de la route vers le repère de coordonnées du véhicule, est le **décalage** du repère de coordonnées de la route vers le repère de coordonnées du véhicule.

Pour cela, **remplacez** l'espace réservé `None` par votre code pour définir une matrice de transformation homogène qui effectue une translation de (x<sub>shift</sub>, y<sub>shift</sub>) par rapport au repère route.

##### __Conseils :__
- La matrice homogène incorporant une translation de (x<sub>shift</sub>, y<sub>shift</sub>) peut être écrite comme

$$ \large M_{\text{shift}} = \begin{bmatrix} 1 & 0 & x_{\text{shift}}\\ 
0 & 1 & y_{\text{shift}} \\ 
0 & 0 & 1 \end{bmatrix} $$




In [ ]:
### START CODE HERE ###
M_shift = None
### END CODE HERE ###


### Changer la direction de l'axe y :

Le deuxième ajustement que vous devrez apporter au mapping **M** est le **miroir** du repère de coordonnées de la route sur l'axe x de sorte que l'axe y des deux repères correspondent.

Pour cela, **remplacez** l'espace réservé `None` par votre code pour définir une matrice de transformation homogène qui effectue le miroir.

##### __Conseils :__
- Avec quelle matrice homogène un point homogène **x** = (x,y,1) doit-il changer le signe de y

$$  \large \begin{bmatrix} x\\ 
-y\\ 
1\end{bmatrix} = M_{\text{direction}}\begin{bmatrix} x\\ 
y\\ 
1\end{bmatrix} $$


In [ ]:
### START CODE HERE ###

M_direction = None

### END CODE HERE ###


### Changer l'échelle de l'axe y :

Le deuxième ajustement que vous devrez apporter au mapping **M** serait l'**échelle** des coordonnées dans le repère route. Cette mise à l'échelle dépend de la résolution spécifiée pour la transformation.

Pour cela, **remplacez** l'espace réservé `None` par votre code pour définir une matrice de transformation homogène qui effectue la mise à l'échelle.

##### __Conseils :__
- Une transformation homogène qui effectue une mise à l'échelle des entrées avec la valeur **a** peut être écrite comme :
$$ \large M_{\text{scale}} = \begin{bmatrix} a & 0 & 0\\ 
0 & a & 0 \\ 
0 & 0 & 1 \end{bmatrix} $$

- Rappelez-vous que les entrées (coordonnées dans l'image BEV) ont l'unité pixel et (coordonnées dans la route) l'unité mètre
- Utilisez `px_per_m` pour la mise à l'échelle.

In [ ]:
### START CODE HERE ###

M_scale = None

### END CODE HERE ###


### Transformation M ajustée :
La nouvelle transformation M combine `M_2Dto3D` et les opérations de **décalage**, **miroir** et **mise à l'échelle** nécessaires pour l'ajustement.

**remplacez** l'espace réservé `None` par votre code pour définir la matrice de transformation ajustée **M**.

#### __Conseils__
- (Un) ordre valide des opérations : décalage (`M_shift`), miroir(`M_direction`), mise à l'échelle(`M_scale`), puis mapping (`M_2Dto3D`)

In [ ]:
### START CODE HERE ###
# direct mapping (assuming z=0)

M_2Dto3D = None
# adjusted
M = None
### END CODE HERE ###



Now that we have all the needed parts to build a mapping between camera images and the BEV image, we can compute the **inverse perspective mapping matrix**. 

Maintenant que nous avons toutes les parties nécessaires pour construire un mapping entre les images de caméra et l'image BEV, nous pouvons calculer la **matrice de transformation de perspective inverse**.

## Tâche : Calculer la matrice de transformation de perspective inverse
Calculez le mapping de l'image de la route vers l'image de la caméra avant, puis calculez la matrice inverse, qui transforme les images de la caméra vers la vue de dessus (ou vue route).

Remplacez l'espace réservé `None` par votre code.
#### __Conseils :__
- le mapping de l'image de la route vers l'image de la caméra avant est défini comme suit (nous avons besoin de l'**inverse**)

$$ \\large x_{cam} = P X = K [I|0] E X= K[R|t] M x_r$$

- __X__ est la coordonnée d'un point dans le repère véhicule
- __x<sub>r</sub>__ est la coordonnée du même point dans le repère route
- __x<sub>cam</sub>__ est la coordonnée du même point en coordonnées d'image
- utilisez `numpy.linalg.inv()` (documentation [ici](https://numpy.org/doc/stable/reference/generated/numpy.linalg.inv.html)) pour inverser la matrice

In [ ]:
### START CODE HERE ###

M_ipm = None
### END CODE HERE ###

print(M_ipm) 

## Tâche : Appliquer la matrice de transformation de perspective inverse sur une image
Ici, vous allez utiliser la matrice M_ipm pour appliquer la transformation de perspective.
Vous pouvez utiliser cv2.warpPerspective() qui prend en entrée une image et une matrice de mapping (3x3).
Cette fonction applique la matrice d'homographie à chaque pixel de l'image d'entrée.
Pour chaque position (x<sub>c</sub>,y<sub>c</sub>) dans l'image d'entrée, (x<sub>r</sub>,y<sub>r</sub>) est calculé.
cv2.warpPerspective() applique également l'interpolation.

Remplacez l'espace réservé `None` par votre code.

#### __Conseils :__
- lisez l'image depuis **"ipm_assets/images/vr_1.png"**. Utilisez [`cv2.imread()`](https://docs.opencv.org/3.4/d4/da8/group__imgcodecs.html)
- utilisez la fonction de transformation de perspective d'OpenCV : `cv2.warpPerspective()`
- sortie attendue

![Expected Output](ipm_assets/images/ipm_invalid.png "ipm output")

In [ ]:
### START CODE HERE ###
# read image
image = None

# use M_ipm to perform the perspective mapping
img_out = None
### END CODE HERE ###

show_image(img_out, "Output")

Le mapping que nous avons conçu ne tient pas pour les pixels au-dessus du niveau du sol.
C'est pourquoi, par exemple, les bâtiments sont mappés incorrectement.

Vous pouvez atténuer cet effet en masquant simplement la partie supérieure de l'image d'entrée (qui ne montre pas la route).

Remplacez les espaces réservés `None` par votre code.


#### __Conseils__ :
- Définissez une matrice de zéros (tableau numpy) **mask**, qui a la hauteur et la largeur de l'image d'entrée. Utilisez **dtype="uint8"**
- Dessinez un rectangle sur le masque qui a la couleur blanc=255 et couvre la partie de l'image que nous ne voulons pas masquer (moitié inférieure)
- Utilisez **cv2.bitwise_and(image, image, mask=mask)** pour appliquer le masque à l'entrée
- Sortie attendue :

![Expected Output](ipm_assets/images/cropped_image.png "Cropped image")

In [ ]:
### START CODE HERE ###

# extract the height and width of the input image 
input_img_height, input_img_width, _ = None
# intialize the mask with np.zeros  
mask = None
# rectangle for the region of intrest
cv2.rectangle(mask, None, None, None,  255, -1)
# apply mask
image = cv2.bitwise_and(image, image, mask=None)

### END CODE HERE ###

# show the image
show_image(image)


## Tâche : Tout mettre ensemble

Pour éviter de refaire tout le processus de calcul de la matrice IPM manuellement pour différentes caméras, nous pouvons mettre tout ce que nous avons fait dans une fonction qui prend comme entrées :

- image
- matrice extrinsèque de la caméra
- matrice intrinsèque de la caméra
- un dictionnaire de configuration (hauteur de sortie, largeur, résolution, etc.)

et sorties :
- image BEV

Remplacez les espaces réservés `None` par votre code.

##### __Conseils :__
- n'oubliez pas de masquer la moitié supérieure de chaque image d'entrée
- sortie attendue :

![Expected Output](ipm_assets/images/output_expected.png "Cropped image")

In [ ]:
# make sure you have loaded extrinsics_dict, intrinsics_dict
E = extrinsics_dict['vr_1']
K = intrinsics_dict['vr_1']
config = {}
config["px_per_m"] = 10 # number of pixels per meter
config["output_width"] = 798
config["output_height"] = 400
# shift to center of output image
config["shift_x"] = config["output_width"] / 2.0 
config["shift_y"] = config["output_height"] /2.0

In [ ]:
def apply_ipm(image, E, K, config):
    # parameters for ipm
    # output resolution
    px_per_m = config["px_per_m"] 
    # output size
    width = config["output_width"]
    height = config["output_height"]
    # shift to center of the left edge of output image
    shift_x = config["shift_x"]
    shift_y = config["shift_y"]
    
    
    input_img_height, input_img_width, _ = image.shape
    
    ### START CODE HERE ###

    mask = None
    cv2.rectangle(None, None, None,  255, -1)
    image = None
    
    # define matrix that maps from the road frame to the vehicle frame
    
    M_2Dto3D = None
    M_direction = None
    M_shift = None
    M_scale = None
    
    M = None
    
    # define projection matrix
    P = None
    
    M_ipm = None
    
    
    img_out = None
    
    ### END CODE HERE ###

    return img_out

In [ ]:
# read image
img = cv2.imread("ipm_assets/images/vr_1.png")
img = img[:, :, :]

print(img.shape)
# show image 
show_image(img, "Original Image")

# apply ipm
img_out2 = apply_ipm(img, E, K, config)
# show output image 
show_image(img_out2, "Output")


L'utilisation d'une seule caméra ne donne pas une vue BEV à 360°. Nous pouvons appliquer l'IPM à des images provenant de caméras orientées dans différentes directions et assembler le résultat en BEV.

## Tâche : Assembler plusieurs images en BEV
Dans cette tâche, vous allez utiliser des images provenant de 8 caméras différentes.

Vous allez charger plusieurs images depuis un dossier et utiliser les paramètres extrinsèques et intrinsèques de la caméra chargés depuis le fichier JSON pour appliquer la transformation de perspective inverse aux images.

Vous allez implémenter une stratégie d'assemblage simple : simplement ajouter les images des différentes caméras en BEV.
Remplacez les espaces réservés `None` par votre code.

#### __Conseils :__
- Sauvegardez les images dans un dictionnaire (par exemple `images_dict = {'vr_1': ...}`) avec la même clé utilisée pour les paramètres de la caméra
- Créez un tableau numpy de zéros qui contiendra l'image BEV totale avec la forme : `config['output_height'], config['output_width'],3)`
- Itérez sur toutes les images : (par exemple `for image_key, image_val in images_dict.items()` ...)
- Ajoutez les images à l'image totale **uniquement aux positions qui sont encore** `= [0, 0, 0 ]` (pixels noirs)
- Les pixels noirs dans une image peuvent être isolés en utilisant "`image[image==(0,0,0)]`"
- sortie attendue :

![Expected Output](ipm_assets/images/rgb_bev_360.png "RGB BEV")

In [ ]:
# load images from folder
images_directory = "ipm_assets/cameras/images"
images = {}
for filename in os.listdir(images_directory):
    if filename.endswith(".jpg"):
        image_pth = os.path.join(images_directory, filename)
        #print(image_pth)
        images[filename.split('.')[0]] = cv2.imread(image_pth)

In [ ]:
# apply bev and add images 
def apply_ipm_and_stitch_images(images, config, extrinsics_dict, intrinsics_dict):
    ### START CODE HERE ###
    bev_total_img = None
    for n, image in images.items():
        output_image = None
        #show_image(output_image, n)
        bev_total_img[bev_total_img==(0,0,0)] = None
    ### END CODE HERE ###

    show_image(bev_total_img)                     
        
apply_ipm_and_stitch_images(images, config, extrinsics_dict, intrinsics_dict)
        

## Tâche : Appliquer l'IPM à des images segmentées sémantiquement
De la même manière que nous avons appliqué l'IPM à des images RGB, nous pouvons **l'appliquer à des images segmentées sémantiquement**. Les **images BEV résultantes seront également segmentées sémantiquement**.

La tâche ici est d'effectuer une cartographie sémantique par grille basée sur caméra en utilisant l'IPM sur des images segmentées sémantiquement.
Ici, vous utiliserez la même approche que nous avons utilisée pour les images de caméra normales.

Exécutez les deux cellules suivantes, puis remplacez les espaces réservés `None` par votre code.

#### __Conseils :__
- sortie attendue :

![Expected Output](ipm_assets/images/sem_bev_360.png "RGB BEV")

In [ ]:
# get extrinsic matrices dictionary
extrinsics_dict = get_extrinsics(f_path="ipm_assets/carla_data/extrinsics_rpy.json")

intrinsics_dict = get_intrinsics(f_path="ipm_assets/carla_data/intrinsics.json")

In [ ]:
# load images from folder
images_directory = "ipm_assets/carla_data/frames"
images = {}
for filename in os.listdir(images_directory):
    if filename.endswith(".jpg"):
        image_pth = os.path.join(images_directory, filename)
        #print(image_pth)
        images[filename.split('.')[0]] = cv2.imread(image_pth)
# list all images names in the dictionary "images"       
print(images.keys())

# show front right image 
show_image(images['front_right_1'])

In [ ]:
### START CODE HERE ###        

apply_ipm_and_stitch_images(None, None, None, None)

### END CODE HERE ###

## Récapitulatif

- Vous avez appris à appliquer la transformation de perspective inverse en utilisant OpenCV.
- Vous avez appris à appliquer la transformation de perspective inverse en utilisant le modèle de caméra et les transformations de coordonnées.
- Vous avez appris à assembler plusieurs images en vue aérienne.
- Vous avez appris à calculer une cartographie sémantique par grille basée sur la géométrie.